In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import sys

In [2]:
from xlrd import *
from xlsxwriter import *

In [3]:
from PyQt5.uic import loadUiType

In [4]:
import datetime

In [5]:
import pymysql

In [6]:
ui,_ = loadUiType('library.ui')

In [7]:
login,_ = loadUiType('login.ui')

In [8]:
class Login(QWidget, login) :
    def __init__(self) :
        
        QWidget.__init__(self)
        self.setupUi(self)
        self.login.clicked.connect(self.Handle_Login)
        self.Dark_Theme()
        
    def Dark_Theme(self) :
        style = open('themes/ManjaroMix.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
        
    def Handle_Login(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        username = self.username.text()
        password = self.password.text()
        sql = '''SELECT * FROM users'''
        

        self.cur.execute(sql)
        data = self.cur.fetchall()
        for row in data :
            if username == row[1] and password == row[3] :
                #print('User Matched')
                self.window2 = MainApp()
                self.close()
                self.window2.show()
            else :
                self.label_2.setText("Make sure you enter your username and password correctly.")  # the text would be displayed in the label

In [9]:
class MainApp(QMainWindow, ui) :
    def __init__(self) :
        QMainWindow.__init__(self)
        self.setupUi(self)
        self.Handle_UI_Changes()
        self.Handle_Buttons()
        self.Dark_Theme()
        
        self.Show_Author()
        self.Show_Category()
        self.Show_Publisher()
        
        self.Show_Category_Combobox()
        self.Show_Author_Combobox()
        self.Show_Publisher_Combobox()
        
        self.Show_All_Operations()
        
    def Handle_UI_Changes(self) :
        self.Hide_Themes()
        self.tabWidget.tabBar().setVisible(False)
    
    def Handle_Buttons(self) :
        self.themes.clicked.connect(self.Show_Themes)
        self.close_2.clicked.connect(self.Hide_Themes)
        
        self.today.clicked.connect(self.Open_Day_To_Day_Tab)
        self.books.clicked.connect(self.Open_Books_Tab)
        self.users.clicked.connect(self.Open_Users_Tab)
        self.clients.clicked.connect(self.Open_Clients_Tab)
        self.settings.clicked.connect(self.Open_Settings_Tab)
        
        self.save.clicked.connect(self.Add_New_Books)
        self.search.clicked.connect(self.Search_Books)
        self.edit_save.clicked.connect(self.Edit_Books)
        self.delete_2.clicked.connect(self.Delete_Books)
        
        self.Show_All_Books()
        self.Show_All_Clients()
        
        self.add_client.clicked.connect(self.Add_New_Clients)
        self.search_client.clicked.connect(self.Search_Client)
        self.save_client.clicked.connect(self.Edit_Clients)
        self.delete_client.clicked.connect(self.Delete_Clients)    
            
        self.add_category.clicked.connect(self.Add_Category)
        self.add_author.clicked.connect(self.Add_Author)
        self.add_publisher.clicked.connect(self.Add_Publisher)
        
        self.add_user.clicked.connect(self.Add_New_User)
        self.login.clicked.connect(self.Login)
        self.edit.clicked.connect(self.Edit_User)
        
        self.pushButton_22.clicked.connect(self.Dark_Blue_Theme)
        self.pushButton_24.clicked.connect(self.Dark_Grey_Theme)
        self.pushButton_23.clicked.connect(self.Dark_Orange_Theme)
        self.pushButton_25.clicked.connect(self.Light_Theme)
        
        self.add.clicked.connect(self.Handle_Day_Operations)
        
        self.export_today.clicked.connect(self.Export_Day_Operations)
        self.export_books.clicked.connect(self.Export_Books)
        self.export_clients.clicked.connect(self.Export_Clients)
    def Show_Themes(self) :
        self.groupBox_4.show()
    
    def Hide_Themes(self) :
        self.groupBox_4.hide()
    
    #########################
    #######opening tabs######
    #########################
    
    def Open_Day_To_Day_Tab(self) :
        self.tabWidget.setCurrentIndex(0)
    
    def Open_Books_Tab(self) :
        self.tabWidget.setCurrentIndex(1)
    
    def Open_Clients_Tab(self) :
        self.tabWidget.setCurrentIndex(2)
    
    def Open_Users_Tab(self) :
        self.tabWidget.setCurrentIndex(3)
    
    def Open_Settings_Tab(self) :
        self.tabWidget.setCurrentIndex(4)
        
    #########################
    #####Day Operations######
    #########################
    
    def Handle_Day_Operations(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        book_title = self.today_book_title.text()
        client_name = self.today_client_name.text()
        type = self.type.currentText()
        days_number = self.days.currentIndex() + 1
        today_date = datetime.date.today()
        to_date = today_date + datetime.timedelta(days = days_number)
        
        #print(today_date)
        #print(to_date)
        
        self.cur.execute('''
            INSERT INTO dailyoperations(book_name, client, type, days_number, today_date, to_date)
            VALUES (%s, %s, %s, %s, %s, %s)
        ''', (book_title, client_name, type, days_number, today_date, to_date))
        self.db.commit()
        self.statusBar().showMessage("New Operation Added")
        self.Show_All_Operations()
        
    def Show_All_Operations(self) :
        
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''
            SELECT book_name, client, type, today_date, to_date FROM dailyoperations
        ''')
        data = self.cur.fetchall()
        #print(data)
        self.tableWidget.setRowCount(0)   #to only show one unfilled row after all the filled rows
        self.tableWidget.insertRow(0)     #to insert row
        for row, form in enumerate(data) :
            for column, item in enumerate(form) :
                self.tableWidget.setItem(row, column, QTableWidgetItem(str(item)))
                column += 1
            row_position = self.tableWidget.rowCount()
            self.tableWidget.insertRow(row_position)
            
    #########################
    #########Books###########
    #########################
    
    def Add_New_Books(self) :
        
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        book_title = self.book_title.text()
        book_description = self.description.toPlainText()
        book_code = self.book_code.text()                   #book_category = self.category.currentIndex()
        book_category = self.category.currentText()         # At first combobox we took inetegr and so for author, publisher and category we used currentIndex()
        book_author = self.author.currentText() 
        book_publisher = self.publisher.currentText()   
        book_price = self.book_price.text()
        
        self.cur.execute( '''
            INSERT INTO book(book_name, book_description, book_code, book_category, book_author, book_publisher, book_price)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
           ''' ,(book_title, book_description, book_code, book_category, book_author, book_publisher, book_price))
        
        self.db.commit()
        self.statusBar().showMessage('New Book Added')
        
        self.book_title.setText('')
        self.description.setPlainText('')
        self.book_code.setText('')
        self.category.setCurrentIndex(0)
        self.author.setCurrentIndex(0)
        self.publisher.setCurrentIndex(0)
        self.book_price.setText('')
        self.Show_All_Books()
        
    def Show_All_Books(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''SELECT book_code, book_name, book_description, book_category, book_author, book_publisher, book_price FROM book''')
        data = self.cur.fetchall()
        
        
        #print(data)
        self.tableWidget_2.setRowCount(0)
        self.tableWidget_2.insertRow(0)
        
        for row, form in enumerate(data) :
            for column, item in enumerate(form) :
                self.tableWidget_2.setItem(row, column, QTableWidgetItem(str(item)))
                column += 1
              
            row_position = self.tableWidget_2.rowCount()
            self.tableWidget_2.insertRow(row_position)
            
        self.db.close()
        
    def Search_Books(self) :

        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        book_title = self.book_title_line.text()
        
        sql = '''SELECT * FROM book WHERE book_name = %s'''
        self.cur.execute(sql, [(book_title)])
        
        data = self.cur.fetchone()
        
        #print(data)
        
        self.book_title_line2.setText(data[1])   # data[1] ---> As book_title is at index 1
        self.edit_description.setPlainText(data[2])
        self.edit_code.setText(data[3])
        self.edit_category.setCurrentText(data[4])
        self.edit_author.setCurrentText(data[5])
        self.edit_publisher.setCurrentText(data[6])
        self.edit_price.setText(str(data[7]))    # price was integer so converting into string.
        
    
    def Edit_Books(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        book_title = self.book_title_line2.text()
        book_description = self.edit_description.toPlainText()
        book_code = self.edit_code.text()
        book_category = self.edit_category.currentText()     #--|
        book_author = self.edit_author.currentText()         #--| As they are combobox we are using integral value.
        book_publisher = self.edit_publisher.currentText()   #--|
        book_price = self.edit_price.text()
        
        
        search_book_title = self.book_title_line.text()
        
        self.cur.execute('''
            UPDATE book SET book_name = %s, book_description = %s, book_code = %s, book_category = %s, book_author = %s, book_publisher = %s, book_price = %s
            Where book_name = %s
            ''', (book_title, book_description, book_code, book_category, book_author, book_publisher, book_price, search_book_title))
        self.db.commit()
        self.statusBar().showMessage("Book Updated")
        self.Show_All_Books()
    
    def Delete_Books(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        book_title = self.book_title_line.text()
        
        # warning box code
        warning = QMessageBox.warning(self, 'Delete Book', "Are you  sure you want to delete this book?", QMessageBox.Yes | QMessageBox.No)
        if warning == QMessageBox.Yes :
            sql = '''DELETE FROM book where book_name = %s'''
            self.cur.execute(sql, [(book_title)])
            self.db.commit()
            self.statusBar().showMessage("Book Deleted")
        
        self.Show_All_Books()
    
    #########################
    #######Clients###########
    #########################
    
    def Show_All_Clients(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''SELECT client_name, client_email, client_nationalid FROM clients''')
        data = self.cur.fetchall()
        
        
        #print(data)
        self.tableWidget_3.setRowCount(0)   #to show only one row unfilled row after the filled rows.
        self.tableWidget_3.insertRow(0)
        
        for row, form in enumerate(data) :
            for column, item in enumerate(form) :
                self.tableWidget_3.setItem(row, column, QTableWidgetItem(str(item)))
                column += 1
              
            row_position = self.tableWidget_3.rowCount()
            self.tableWidget_3.insertRow(row_position)
            
        self.db.close()
    
    def Add_New_Clients(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        client_name = self.client_name.text()
        client_email = self.client_email.text()
        client_nationalid = self.client_id.text()
    
        self.cur.execute('''
            INSERT INTO clients(client_name, client_email, client_nationalid)
            VALUES (%s, %s, %s)
        ''', (client_name, client_email, client_nationalid))
        
        self.db.commit()
        self.db.close()
        self.statusBar().showMessage("New Client Added")
        self.Show_All_Clients()   # to add the newly added data to show all clients tab
        
    
    def Search_Client(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        client_national_id = self.edit_client_id.text()
        sql = '''SELECT * FROM clients WHERE client_nationalid = %s'''
        self.cur.execute(sql, [(client_national_id)])
        data = self.cur.fetchone()
        #print(data)
        self.edit_client_name.setText(data[1])
        self.edit_client_email.setText(data[2])
        self.edit_client_id2.setText(data[3])
    
    def Edit_Clients(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        client_original_national_id = self.edit_client_id.text()
        client_name = self.edit_client_name.text()
        client_email = self.edit_client_email.text()
        client_national_id = self.edit_client_id2.text()
        
        self.cur.execute('''
            UPDATE clients SET client_name = %s, client_email = %s, client_nationalid = %s WHERE client_nationalid = %s
        ''', (client_name, client_email, client_national_id, client_original_national_id))
        self.db.commit()
        self.statusBar().showMessage("Client Data Updated")
        self.Show_All_Clients()  #to add edited data to show all clients tab
    
    def Delete_Clients(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        client_original_national_id = self.edit_client_id.text()
        
        warning_message = QMessageBox.warning(self, "Delete Client", "Are you sure you want to Delete this client?", QMessageBox.Yes | QMessageBox.No)
        if warning_message == QMessageBox.Yes :

            sql = ''' DELETE FROM clients WHERE client_nationalid = %s'''
            self.cur.execute(sql, [(client_original_national_id)])

            self.db.commit()
            self.db.close()
            self.statusBar().showMessage("Client Deleted")
            self.Show_All_Clients()   #to delete the data of the client from the show all clients tab
    
    #########################
    #######Users#############
    #########################
    
    def Add_New_User(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        username = self.username.text()
        email = self.email.text()
        password = self.password.text()
        password2 = self.password2.text()
        
        if password == password2 :
            self.cur.execute('''
                INSERT INTO users(user_name, user_email, user_password)
                VALUES (%s, %s, %s)
            ''',(username, email, password))
            
            self.db.commit()
            self.statusBar().showMessage("New User Added")
            
        else :
            self.label_29.setText("Please Add A Valid Password Twice")
    
    def Login(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        username = self.login_username.text()
        password = self.login_password.text()
        
        sql = '''SELECT * FROM users'''
        

        self.cur.execute(sql)
        data = self.cur.fetchall()
        for row in data :
            if username == row[1] and password == row[3] :
                print('User Matched')
                self.statusBar().showMessage("Valid Username and Password.")
                self.groupBox_3.setEnabled(True)
                
                self.edit_username.setText(row[1])
                self.edit_email.setText(row[2])
                self.edit_password.setText(row[3])
                #self.edit_password2.setText(row[4])
            else :
                self.statusBar().showMessage("Enter a valid username or password.")
            
    
    def Edit_User(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        username= self.edit_username.text()
        email = self.edit_email.text()
        password = self.edit_password.text()
        password2 = self.edit_password2.text()
        
        original_name = self.login_username.text()
        
        #print(username, email, password) ---> the original username would be changed to new one.
        
        if password == password2 :
            self.cur.execute(''' 
            UPDATE users SET user_name = %s, user_email = %s, user_password = %s WHERE user_name = %s 
            ''', (username, email, password, original_name))  # original_name is the previous username.
            self.db.commit()
            self.statusBar().showMessage("User Data Updated Successfully")
            
        else :
            print("Passwords do not match.")
    
    #########################
    #######Settings##########
    #########################
    
    def Add_Category(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        category_name = self.category_name.text()
        self.cur.execute('''
            INSERT INTO category (category_name) VALUES (%s)
        ''' , (category_name,))
        
        self.db.commit()
        #print("done")
        self.statusBar().showMessage("New Category Added")
        self.category_name.setText('')
        self.Show_Category()
        self.Show_Category_Combobox()
        
    def Show_Category(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute(''' SELECT category_name from category''')
        data = self.cur.fetchall()
        
        #print(data)
        
        if data :
            self.category_table.setRowCount(0)
            self.category_table.insertRow(0)
            for row, form in enumerate(data) :
                for  column, item in enumerate(form) :
                    self.category_table.setItem(row, column, QTableWidgetItem(str(item)))
                    column += 1
                row_position = self.category_table.rowCount()
                self.category_table.insertRow(row_position)
                
    def Add_Author(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        author_name = self.author_name.text()
        self.cur.execute('''
            INSERT INTO authors (author_name) VALUES (%s)
        ''' , (author_name,))
        
        self.db.commit()
        self.author_name.setText('')
        #print("done")
        self.statusBar().showMessage("New Author Added")
        self.Show_Author()
        self.Show_Author_Combobox()
        
    def Show_Author(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute(''' SELECT author_name from authors''')
        data = self.cur.fetchall()
        
        #print(data)
        
        if data :
            self.author_table.setRowCount(0)
            self.author_table.insertRow(0)
            for row, form in enumerate(data) :
                for  column, item in enumerate(form) :
                    self.author_table.setItem(row, column, QTableWidgetItem(str(item)))
                    column += 1
                row_position = self.author_table.rowCount()
                self.author_table.insertRow(row_position)
    
    
    def Add_Publisher(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        publisher_name = self.publisher_name.text()
        self.cur.execute('''
            INSERT INTO publisher (publisher_name) VALUES (%s)
        ''' , (publisher_name,))
        
        self.db.commit()
        self.publisher_name.setText('')
        #print("done")
        self.statusBar().showMessage("New Publisher Added")
        self.Show_Publisher()
        self.Show_Publisher_Combobox()
        
    def Show_Publisher(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()

        self.cur.execute(''' SELECT publisher_name from publisher''')
        data = self.cur.fetchall()

        #print(data)

        if data :
            self.publisher_table.setRowCount(0)
            self.publisher_table.insertRow(0)
            for row, form in enumerate(data) :
                for column, item in enumerate(form) :
                    self.publisher_table.setItem(row, column, QTableWidgetItem(str(item)))
                    column += 1
                row_position = self.publisher_table.rowCount()
                self.publisher_table.insertRow(row_position)
                
    #########################
    ###Show Settings in Ui###
    #########################
    
    def Show_Category_Combobox(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute( ''' SELECT category_name FROM category ''')
        data = self.cur.fetchall()
        
        self.category.clear()
        for category in data :
            #print(category[0])
            self.category.addItem(category[0])  # category ---> name of combo box
            self.edit_category.addItem(category[0])   # in the edit book tab we are adding the category of the particular book we
                                                        # we searched for
            
    def Show_Author_Combobox(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute(''' SELECT author_name FROM authors''')
        data = self.cur.fetchall()
        
        for author in data :
            self.author.addItem(author[0])     # author ---> name of combo box
            self.edit_author.addItem(author[0])  # in the edit book tab we are adding the author of the particular book we
                                                        # we searched for
            
    def Show_Publisher_Combobox(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute(''' SELECT publisher_name FROM publisher''')
        data = self.cur.fetchall()
        
        for publisher in data :
            self.publisher.addItem(publisher[0])    # publisher ---> name of combo box
            self.edit_publisher.addItem(publisher[0])  # in the edit book tab we are adding the publisher of the particular book we
                                                        # we searched for
                
    
    ##########################
    #######Export Data########
    #########################
    
    def Export_Day_Operations(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''
            SELECT book_name, client, type, today_date, to_date FROM dailyoperations
        ''')
        
        data = self.cur.fetchall()
        wb = Workbook('day_operations.xlsx')  #created a workbook with this name.
        sheet1 = wb.add_worksheet()   #cretaed sheet where we have saved this file.
        
        sheet1.write(0, 0, 'Book Title')  #we are taking the row number 0 and column no. 1
        sheet1.write(0, 1, 'Client Name')
        sheet1.write(0, 2, 'Type')
        sheet1.write(0, 3, 'From Date')
        sheet1.write(0, 4, 'To Date')
        
        row_number = 1  #in 0th row there would be the heading so starting from row 1 now.
        for row in data :
            column_number = 0 #starting from column 0 of row 1
            for item in row :
                sheet1.write(row_number, column_number, str(item))
                column_number += 1
            row_number += 1   
        self.statusBar().showMessage("Day Operation Report Created Successfully.")
        
        wb.close()
    
    def Export_Books(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''SELECT book_code, book_name, book_description, book_category, book_author, book_publisher, book_price FROM book''')
        data = self.cur.fetchall()
        
        wb = Workbook("Books.xlsx")
        sheet1 = wb.add_worksheet()
        
        sheet1.write(0, 0, 'Book Code')
        sheet1.write(0, 1, 'Book Dame')
        sheet1.write(0, 2, 'Book Description')
        sheet1.write(0, 3, 'Book Category')
        sheet1.write(0, 4, 'Book Author')
        sheet1.write(0, 5, 'Book Publisher')
        sheet1.write(0, 6, 'Book Price')
        
        row_number = 1
        for row in data :
            column_number = 0
            for item in row :
                sheet1.write(row_number, column_number, str(item))
                column_number += 1
            row_number += 1
        self.statusBar().showMessage("Books Report Created Successfully.")
        
        wb.close()
        
    
    def Export_Clients(self) :
        self.db = pymysql.connect("localhost", "root", "", "library")
        self.cur = self.db.cursor()
        
        self.cur.execute('''SELECT client_name, client_email, client_nationalid FROM clients''')
        data = self.cur.fetchall()
        
        wb = Workbook("Clients.xlsx")
        sheet1 = wb.add_worksheet()
        
        sheet1.write(0, 0, 'Client Name')
        sheet1.write(0, 1, 'Client Email')
        sheet1.write(0, 2, 'Client National Id')
        
        row_number = 1
        for row in data :
            column_number = 0
            for item in row :
                sheet1.write(row_number, column_number, str(item))
                column_number += 1
            row_number += 1
        self.statusBar().showMessage("Clietns report Created Successfully")
        
        wb.close()
            
    
    
    #########################
    #######Ui Themes#########
    #########################
    
    def Dark_Blue_Theme(self) :
        style = open('themes/darkblue.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
    
    def Dark_Grey_Theme(self) :
        style = open('themes/darkgray.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
    
    def Dark_Orange_Theme(self) :
        style = open('themes/darkorange.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
    
    def Light_Theme(self) :
        style = open('themes/light.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
        
    def Dark_Theme(self) :
        style = open('themes/ManjaroMix.css', 'r')
        style = style.read()
        self.setStyleSheet(style)
    
def main() :
    app = QApplication(sys.argv)
    window = Login()
    window.show()
    app.exec_()

if __name__ == "__main__" :
    main()